In [ ]:
###Load modules
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By

# creating a new Chrome driver service
chrome_driver_service = webdriver.chrome.service.Service('C:/Users/Banty/Desktop/Information-Architectures/chromedriver.exe')

# Starting the chrome service
chrome_driver_service.start()

# Creating a new Chrome browser instance using the service
chrome_browser = webdriver.Chrome(service=chrome_driver_service)

# enter the URL path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

# Identify xpath of location to select element
inputElement = browser.find_element(By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()
time.sleep(4)  # Adding a sleep element that will allow for the page to load

# Identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR, 'table.Bordered')
time.sleep(1)

# Create empty dataframe
df_list = []

# Loop through pages of results using while loop
while True:
    # Loop through dataframe to export table
    for row in table.find_elements(By.CSS_SELECTOR, 'tr'):
        cols = [cell.text for cell in row.find_elements(By.CSS_SELECTOR, 'td')]
        if len(cols) > 0:  # Excluding empty rows 
            df_list.append({"Organization Name": cols[0], "NY Reg #": cols[1], "EIN": cols[2], "Registrant Type": cols[3], "City": cols[4], "State": cols[5]})

    # Check if there is another page for the results
    next_button = browser.find_elements(By.XPATH, "//a[contains(text(),'Next')]")
    if len(next_button) > 0:
        next_button[0].click()
        time.sleep(4)  # Adding a sleep element that will allow for the page to load
        table = browser.find_element(By.CSS_SELECTOR, 'table.Bordered')
    else:
        break

# Integrating all dataframes into a single dataframe
df = pd.concat([pd.DataFrame(x, index=[0]) for x in df_list], ignore_index=True)

# Displaying the scraped data
display(df)

In [ ]:
import boto3
from botocore.exceptions import ClientError

# Assigning the bucket name against the bucket variable
bucket_name = 'm10-assignments'

aws_access = input('Enter your access key')
aws_secret = input('Enter your secret key')

aws_s3_client = boto3.client('s3',
          aws_access_key_id = aws_access,
          aws_secret_access_key = aws_secret)

# Below code has the access key to my AWS account 
response = aws_s3_client.list_buckets()
bucket_exist = False

for bucket in response['Buckets']:
    if bucket['Name'] == bucket_name:
        bucket_exist = True
        break

if bucket_exist:
    print("The bucket exists")
else:
    print("The bucket does not exist")

# Create the bucket if it doesn't exist
if not bucket_exist:
    try:
        aws_s3_client.create_bucket(Bucket=bucket_name)
        print(f"{bucket_name} Bucket has been created on AWS S3")
    except ClientError as e:
        print(e)
        print(f"{bucket_name} Can't be created on S3")
    except:
        print(f"{bucket_name} Can't be created on S3")

In [ ]:
from io import StringIO
def upload_s3(df,i):
    global aws_s3_client,bucket_name
    csv_buffer = StringIO()
    df.to_csv(csv_buffer,header=True,line_terminator='\n')
    csv_buffer.seek(0)
    aws_s3_client.put_object(Bucket=bucket_name,Body=csv_buffer.getvalue(),Key=i)
    
filename= 'charities_bureau_scrape_group1_part2'# Name of the group
datetime = time.strftime("%Y%m%d%H%M%S") # Timestamp
filenames3 = "%s%s.csv"%(filename,datetime) # Name of the filepath in S3 and csv file

# Checking if the bucket exists
response = aws_s3_client.list_buckets()
bucket1 = [bucket['Name'] for bucket in response['Bucket1']]
if bucket_name not in buckets:
    print(f"{bucket_name} bucket does not exist.")
else:
    print("Uploading Data")
    upload_s3(df, filenames3)
    print("Data uploaded successfully ")



#print the success message
print("Successfull uploaded file to location:"+str(filenames3))